!wget https://raw.githubusercontent.com/mesolitica/malaysian-dataset/master/llm-benchmark/BM-pt3/BM-A-pt3

In [1]:
import os

# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
#mesolitica/malaysian-llama2-7b-32k-instructions-v2
tokenizer = AutoTokenizer.from_pretrained('mesolitica/malaysian-mistral-7b-32k-instructions')
# model = AutoModelForCausalLM.from_pretrained(
#     'fpf-7b-instructions-16k-call-v2/checkpoint-9300', 
#     use_flash_attention_2 = True, 
#     torch_dtype = torch.float16,
#     device_map="cuda:0"
# )

model = AutoModelForCausalLM.from_pretrained(
    'mesolitica/malaysian-mistral-7b-32k-instructions', 
    #use_flash_attention_2 = True, 
    torch_dtype = torch.float16,
    device_map="auto"
)

/home/cybertron/anaconda3/envs/sgpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████| 3/3 [00:13<00:00,  4.66s/it]


In [3]:
from tqdm import tqdm
import os
import random
import json
random.seed(404)
torch.manual_seed(404)

data_folder = '/media/cybertron/fa54fcb6-b5e1-492e-978a-6389519c168a/llm-benchmarks/BM-PT3/data/'
with open(data_folder + 'BM-A-pt3') as fopen:
    text = fopen.read()
print(text.split('no: '))

In [4]:

data_folder = '/media/cybertron/fa54fcb6-b5e1-492e-978a-6389519c168a/llm-benchmarks/sejarah-spm/data/'
with open(data_folder+ 'sejarah_spm') as fopen:
    text = fopen.read()
    
questions = []
for t in text.split('no: ')[1:]: # split by "no"
    t = t.strip()
    no = t.split('\n')[0] # "split by newline to get index'
    soalan = t.split('soalan:')[1].split('jawapan:')[0].strip()
    jawapan = t.split('jawapan: ')[1].split(',')[0].strip()
    data = {
        'no': no,
        'soalan': soalan,
        'jawapan': jawapan,
    }
    questions.append(data)

In [5]:
questions[0]

{'no': '1',
 'soalan': 'Apakah yang dimaksudkan dengan wilayah pengaruh?\nA. Kawasan tanah jajahan dan takluk raja\nB. Kawasan yang dinaungi oleh raja dan kerabatnya\nC. Kawasan yang rakyatnya menerima dan memperakui pemerintahan seseorang raja\nD. Kawasan yang rakyatnya menganggap raja sebagai tuhan atau wakil tuhan',
 'jawapan': 'C'}

In [6]:
arange = set(range(len(questions)))
print(arange)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}


In [7]:
def convert_prompt(row, answer = False): # convert question to prompt 
    if answer: # include answer in prompt
        prompt = f""" 
soalan: {row['soalan']}
jawapan: {row['jawapan']}
    """
    else: # do not include answer in prompt
        prompt = f"""
soalan: {row['soalan']}
jawapan:
    """
    return prompt.strip()

def parse_mistral_chat(messages):

    user_query = messages[-1]['content']

    users, assistants = [], []
    for q in messages[:-1]:
        if q['role'] == 'user':
            users.append(q['content'])
        elif q['role'] == 'assistant':
            assistants.append(q['content'])

    texts = ['<s>']
    for u, a in zip(users, assistants):
        texts.append(f'[INST] {u.strip()} [/INST]{a.strip()}</s> ')

    texts.append(f'[INST] {user_query.strip()} [/INST]')
    prompt = ''.join(texts).strip()
    return prompt

In [8]:
for i in tqdm(range(len(questions))):
    
    prompts = []
    prompts.append(convert_prompt(questions[i])) # convert the question to prompt
    prompt = '\n\n'.join(prompts)
    messages = [
        {'role': 'user', 'content': prompt}
    ]
    prompt = parse_mistral_chat(messages)
    inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda') # get input ids 
    repeat = []
    for _ in range(5): # for each question repeat 5 times
        try:
            generate_kwargs = dict(
                inputs,
                max_new_tokens=50,
                top_p=0.95,
                top_k=50,
                temperature=0.9,
                do_sample=True,
                num_beams=1,
                repetition_penalty=1.05,
            )
            r = model.generate(**generate_kwargs)
            splitted = tokenizer.decode(r[0]).split('[/INST]')[1].strip().replace('</s>', '').replace('.', '').replace(',', '').strip().split()
            splitted = [t for t in splitted if len(t) == 1 and t in {'A', 'B', 'C', 'D'}]
            repeat.append(splitted[0])
    
        except Exception as e:
            pass
    
    questions[i]['output'] = repeat

  0%|                                                    | 0/20 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  5%|██▏                                         | 1/20 [00:01<00:37,  1.98s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 10%|████▍                                       | 2/20 [00:03<00:29,  1.63s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad

In [9]:
def most_common(l):
    return max(set(l), key=l.count)

In [10]:
filtered = [q for q in questions if 'output' in q and len(q['output'])]
correct = 0
for q in filtered:
    correct += most_common(q['output']) == q['jawapan']
(correct / len(filtered)) * 100

60.0

In [11]:
del model, tokenizer